# Load all the pig dicom data and convert them to nifti

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# !pip install loguru pydicom scipy scikit-image

# !pip install git+https://github.com/mjirik/io3d --upgrade --force-reinstall
# !pip install git+https://github.com/mjirik/imma --upgrade --force-reinstall

# !pip install git+https://github.com/mjirik/io3d
# !pip install git+https://github.com/mjirik/imma



In [ ]:
import io3d
from pathlib import Path
from loguru import logger
from pprint import pprint, pformat
import pandas as pd
import re
import numpy as np
import json
import tqdm

logger.enable("io3d")
force = True
# force = False

recreate_meta = False
recreate_meta = True
# base_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_dicom")
# dirname = "Prasata DC"
# dirname = "Prasata H"
dataset_base_path = Path(r"~/mnt/nas-bmc3_ct/").expanduser().resolve()
# base_path = dataset_base_path / dirname
base_path = dataset_base_path
# fn_prefix = dirname.replace(" ", '_') + "-"
# output_dir_part = dirname.replace(" ", '_')
# base_path = Path(r"~/Downloads/_temp/").expanduser()  #: used when the zip files are downloaded manually
# raw_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_nii_transposed")
# transposed_path = Path(r"H:\biomedical\orig\pilsen_pigs_all\transplantation_nii_transposed")
# output_path = Path(r"H:\biomedical\orig\pilsen_pigs")
# output_path= Path(r"~/mnt/nas-bmc3_ct/pilsen_pigs_all/transplantation_nii_transposed").expanduser() / output_dir_part
metadata_path=dataset_base_path / "metadata.csv"

assert base_path.exists()

# Convert all to nii

In [ ]:

def touch_file(pth:Path):
    try:
        with open(pth, "rb") as f:
            # read just some part of the file
            f.read(1)
    except Exception as e:
        import traceback
        print(f"Error in touching file {pth}: {e}")
        traceback.print_exc()

In [ ]:
from typing import Union

def get_projection(
        datap:io3d.image.DataPlus, axis:Union[int,str], method:str="max"
):
    """Get projection of 3D data to 2D."""
    if isinstance(axis, str):
        dict_axis = {"axial": 0, "coronal": 1, "sagittal": 2}
        if axis in dict_axis:
            axis = dict_axis[axis]
        else:
            raise ValueError(f"Unknown axis {axis}, use one of {list(dict_axis.keys())} or 0, 1, 2")

    data3d = datap.data3d
    axcodes = datap.orientation_axcodes
    data3d = io3d.image.transform_orientation(data3d, axcodes, "SPL")
    if method == "max":
        data2d = data3d.max(axis=axis)
    elif method == "mean":
        data2d = data3d.mean(axis=axis)
    else:
        raise ValueError(f"Unknown method {method}")
    return data2d



In [ ]:
# data2d = get_projection(datap, 0, "max")
# from matplotlib import pyplot as plt
# plt.style.use('classic')
# plt.imshow(data2d, cmap="gray_r")
# plt.colorbar()
# plt.style.available


In [ ]:
import tqdm

In [ ]:
from pathlib import Path
import tqdm

# List all directories under base_path (recursively)
all_dirs = [p for p in base_path.rglob("*") if p.is_dir() and "Prasata " in str(p)]

leaf_dirs = []
for d in tqdm.tqdm(all_dirs, desc="Checking dirs"):
    children = list(d.iterdir())
    has_subdir = any(child.is_dir() for child in children)
    has_file = any(child.is_file() for child in children)
    if not has_subdir and has_file:
        leaf_dirs.append(d)

leaf_dirs



# leaf_dirs = []
# for d in tqdm.tqdm(list(base_path.glob("**/Prasata */**/"))):
#     if d.is_dir():
#         # Get direct children
#         children = list(d.iterdir())
#         # Identify if there are any subdirectories
#         has_subdir = any(child.is_dir() for child in children)
#         # Identify if there is at least one file in the directory
#         has_file = any(child.is_file() for child in children)
#         # A leaf directory has no subdirectories but does contain files
#         if not has_subdir and has_file:
#             leaf_dirs.append(d)
#
# leaf_dirs

In [ ]:
str(leaf_dirs[0].relative_to(base_path))

In [ ]:
# io3d.read(sorted(leaf_dirs)[0], orientation_axcodes="IPL")

In [ ]:
dir_path = sorted(leaf_dirs)[0]




In [ ]:

current_fn = sorted(leaf_dirs)[0]

def get_series_description(dcm_info, series_number):
    series_description = ""
    if "SeriesDescription" in dcm_info:
        series_description = " " + dcm_info["SeriesDescription"]

    return str(series_number) + series_description

def get_stats_of_series_in_dir(dir_path:str):
    dicomdirectory = io3d.dcmreaddata.DicomDirectory(
        str(dir_path)
    )
    return dicomdirectory.get_stats_of_series_in_dir()

def get_stats_of_series_in_dir_nice_list(current_fn) -> list:
    series_metadata = get_stats_of_series_in_dir(current_fn)
    for kkey in series_metadata:
        if "dcmfilelist" in series_metadata[kkey]:
            # dir_info = series_metadata
            series_metadata[kkey]["dcmfilelist_len"] = len(series_metadata[kkey]["dcmfilelist"])
            series_metadata[kkey]["dcmfilelist"] = None


        if "SeriesNumber" in series_metadata[kkey]:
            series_metadata[kkey]["SeriesNumber"] = int(series_metadata[kkey]["SeriesNumber"])
        if "Count" in series_metadata[kkey]:
            series_metadata[kkey]["Count"] = int(series_metadata[kkey]["Count"])
        series_metadata[kkey]["Path"] = str(current_fn)
    series_metadata_list = list(series_metadata.values())
    return series_metadata_list


metadata_list = get_stats_of_series_in_dir_nice_list(current_fn)
# metadata_list


In [ ]:
from collections.abc import Iterable
df = pd.DataFrame(metadata_list)
# df['voxelsize_mm_0'] = df.voxelsize_mm.apply(lambda x: json.loads(x)[0] if isinstance(x, str) else x[0] if isinstance(x, list) else np.nan)
# df['voxelsize_mm_1'] = df.voxelsize_mm.apply(lambda x: json.loads(x)[1] if isinstance(x, str) else np.nan)
# df['voxelsize_mm_2'] = df.voxelsize_mm.apply(lambda x: json.loads(x)[2] if isinstance(x, str) else np.nan)
# df['voxelsize_mm_2'] = df.voxelsize_mm.apply(lambda x: x[2] if len(x) == 3 else np.nan)
df['voxelsize_mm_0'] = df.voxelsize_mm.apply(lambda x: x[0] if isinstance(x,Iterable) else np.nan)
df['voxelsize_mm_1'] = df.voxelsize_mm.apply(lambda x: x[1] if isinstance(x,Iterable) else np.nan)
df['voxelsize_mm_2'] = df.voxelsize_mm.apply(lambda x: x[2] if isinstance(x,Iterable) else np.nan)
df

In [ ]:
# to utf-8 sig

# df.to_csv(metadata_path, index=False, encoding="utf-8-sig")

In [ ]:
metadata_list = []
for current_fn in sorted(leaf_dirs):
    metadata_ith = get_stats_of_series_in_dir_nice_list(current_fn)
    metadata_list.extend(metadata_ith)


In [ ]:
len(metadata_list)

In [ ]:
# join csvs
def append_dataframe_to_csv(df: pd.DataFrame, csv_filename: Path):
    if csv_filename.exists():
        existing_df = pd.read_csv(csv_filename)
        # Combine using union of columns
        combined_df = pd.concat([existing_df, df], ignore_index=True, sort=False)
    else:
        combined_df = df

    # Save the combined data (overwriting the file)
    combined_df.to_csv(csv_filename, index=False)


append_dataframe_to_csv(df, metadata_path)


In [ ]:
## STOP Here

In [ ]:
# # fnlist = list(base_path.glob("*Tx0*D_V*"))
# # find a leaf directory in base_path which contains some file
#
#
#
# # fnlist = sorted(list(base_path.glob("*Tx0*D_A*")) + list(base_path.glob("*Tx0*D_V*")))[::-1]
# fnlist = sorted(leaf_dirs)
#
#
# print(f"Number of files: {len(fnlist)}")
# from joblib import Parallel, delayed
# import tqdm
# #
# for fn in tqdm.tqdm(fnlist[4:]):
#     # iterate over series
#
#     # dir_info = io3d.dcmreaddata.dicomdir_info(fn, gui=False)
#     dir_info = get_stats_of_series_in_dir(fn)
#
#
#     logger.debug(str(dir_info.keys()))
#
#     for series_number in dir_info.keys():
#         if series_number == None:
#             series_number = "first"
#         # series_description = get_series_description(dir_info, series_number).replace(" ", "_")
#
#         # i want to have there the parent directory
#         # fn_name = str(fn.relative_to(base_path)).replace("\\", "-").replace("/", "-")
#         fn_name = (fn_prefix + str(fn.relative_to(base_path))).replace("\\", "-").replace("/", "-").replace(" ", "_")
#         fn_name += f"_{series_number}"
#
#         # logger.info(fn)
#         fn_in = fn
#         # fn_out = raw_path / fn.name / f"{fn.name}.mhd"
#         fn_out = output_path / fn_name / f"{fn_name}.nii.gz"
#
#         fn_out.parent.mkdir(parents=True, exist_ok=True)
#         fn_meta = fn_out.parent / "meta.json"
#         if force or (not fn_out.exists()):
#
#             try:
#                 fn_fns = sorted(list(fn.glob("*")))
#                 tqdm.tqdm.write(f"Reading series {series_number} with {len(fn_fns)} files in {fn_in} and writing {fn_out}")
#                 # tqdm.tqdm.write(f"Number of files in the directory: {len(fn_fns)}")
#                 # Parallel(n_jobs=2)(delayed(touch_file)(fn) for fn in tqdm.tqdm(fn_fns, desc="touching files"))
#
#                 axcodes = "IPL"
#                 # logger.debug(f"Reading {fn_in} with axcodes={axcodes}")
#                 datap = io3d.read(fn_in,
#                                   # series_number="first",
#                                   series_number=series_number,
#                                   orientation_axcodes=axcodes)
#                 # logger.debug(datap.keys())
#                 io3d.write(datap, fn_out)
#                 # logger.debug("writing done, creating projections")
#                 for axis in ["axial", "coronal", "sagittal"]:
#                     data2d = get_projection(datap, axis, "max")
#                     import skimage.io
#                     # change intensity to range 0..1
#
#                     data2d = (255 * (data2d - np.min(data2d)).astype(float) / (np.max(data2d) - np.min(data2d))).astype(np.uint8)
#                     skimage.io.imsave(fn_out.parent / f"{fn_out.stem}_{axis}.jpg", data2d)
#                 # logger.debug("projections done")
#             except Exception as e:
#                 import traceback
#                 logger.error(f"Error in reading {fn_in}: {e}")
#                 traceback.print_exc()
#                 # logger.debug(f"shape={datap.data3d.shape}, {datap.orientation_axcodes}")
#             # with open(fn_meta, "w") as f:
#             #     json.dump(dict(row), f)
#
#
#
